In [2]:
import pandas as pd, json, matplotlib.pyplot as plt, re
from pdb import set_trace
%matplotlib inline

In [5]:
ls *.csv

ls: cannot access '*.csv': No such file or directory


In [6]:
sc = pd.read_csv('data/scenario.csv')

In [7]:
df = pd.read_csv('data/data_by_sphere.csv')

In [50]:
sum_by_sphere = df.loc[df.year == 2012].groupby('sphere', as_index=False).k_t_ne.sum()
sum_by_sphere.columns = ['sphere', 'total_ktne']

In [4]:
df

,source,is_vde,sphere,year,k_t_ne
0,Газ,0,Всього,2012,21698.0
1,Теплова енергія,0,Всього,2012,11853.0
2,Нафта та нафтопродукти,0,Всього,2012,11347.0
3,Електроенергія,0,Всього,2012,11130.0
4,Вугілля,0,Всього,2012,9212.0
5,Біопаливо та відходи,1,Всього,2012,1029.0
6,Електроенергія з ВДЕ,1,Всього,2012,710.0
7,Теплова енергія з ВДЕ,1,Всього,2012,12.3
8,Сонячна енергія,1,Всього,2012,0.0
9,Газ,0,Всього,2035,7615.0


In [51]:
df = df.merge(sum_by_sphere, on='sphere', how='left')
df.sphere = df.sphere.apply(str.capitalize)
df.sphere = df.sphere.str.replace('Total', 'Всього')

In [52]:
df.sort_values(['total_ktne', 'sphere', 'year', 'is_vde', 'k_t_ne'], ascending=[False, True, True, True, False]
              ).drop('total_ktne', axis=1
              ).to_csv('data_by_sphere.csv', index=False)

In [46]:
measures = {
    'ДОМОГОСПОДАРСТВА': [
        'Термомодернізація житлових будинків',
        'Модернізація систем енергозабезпечення об’єктів ЖКГ',
        'Автоматизовані індивідуальні теплові пункти (ІТП)',
        'Впровадження когенераційних систем для багатоповерхових будинків',
        'Техніка А++, А+++',
        '40-50% населення використовує сонячні панелі на дахах будинків',
        'Опалення та гаряча вода за рахунок біопалива та відходів, централізованого ВДЕ-тепла та використання сонячних колекторів',
    ],
    'ТРАНСПОРТ': [
        'Легкові авто: 90% електромобілі, 10% на біопаливі',
        'Електроавтобуси та електропотяги',
        'Авіаційний, водний, вантажний транспорт на біопаливі',
    ],
    'ПРОМИСЛОВІСТЬ': [
        '88% - енергія з відновлюваних джерел та альтернативних видів палива',
        'Електрифікація промисловості',
        'Заміна енергоємного обладнання на нове та ефективне',
        'Більші інвестиції з боку промислових виробників, зокрема електроенергетичної галузі',
    ],
    'СІЛЬСЬКЕ ГОСПОДАРСТВО': [
        'Використання ВДЕ для виробництва електроенергії та тепла',
        'СГ Транспорт на біопаливі',
        'Вирощування енергетичних культур',
        'Нові технології використання сільськогосподарських машин та механізмів',
    ],
    'СФЕРА ПОСЛУГ': [
        'ВДЕ 88%',
        'Заходи з енергоефективності та енергозаощадження',
        'Термоізоляція приміщень',
        'Використання високоефективних технічних приладів (А++, А+++)',
    ],
    'Всього': [
        'Економічні наслідки:',
        'Скорочення потреб в енергоресурсах на 42%',
        'Економія до 75% енергоресурсів для опалення завдяки повній реконструкції будинків',
        'Економія 30% коштів завдяки енергоефективності та енергозбереженню',
        'Енергоємність ВВП України знизиться у 3  рази',
        'Не потрібно імпортувати газ, вугілля, ядерне паливо',
        'Соціальні наслідки:',
        'Нова енергетична інфраструктура',
        'Нові робочі місця в галузі ВДЕ та енергоефективності',
        'Нові можливості для розвитку на місцях: локальне виробництво енергії, використання місцевих ресурсів',
        'Зменшення кількості працівників у видобувній галузі',
        'Скорочення викидів парникових газів 90 %  від рівня 1990 р.',
        'Зниження негативного впливу від енергетики на здоров’я людей',
    ]
}

In [18]:
def reshape_df(d):
    new = d.pivot('year', 'source', 'k_t_ne').reset_index()
    new['sphere'] = d.sphere.iloc[0]
    return new.reindex(['year', 'sphere', 'Біопаливо та відходи', 'Вугілля', 'Газ', 'Електроенергія',
       'Електроенергія з ВДЕ', 'Нафта та нафтопродукти', 'Сонячна енергія',
       'Теплова енергія', 'Теплова енергія з ВДЕ'], axis=1)

In [25]:
df.groupby(['year', 'sphere'], as_index=False).apply(reshape_df).reset_index(drop=True).fillna(0)

source,year,sphere,Біопаливо та відходи,Вугілля,Газ,Електроенергія,Електроенергія з ВДЕ,Нафта та нафтопродукти,Сонячна енергія,Теплова енергія,Теплова енергія з ВДЕ
0,2012,Всього,1029.0,9212.0,21698.0,11130.0,710.0,11347.0,0.0,11853.0,12.3
1,2012,Домогосподарства,936.0,715.0,13760.0,3105.0,198.0,71.0,0.0,4677.0,5.0
2,2012,Промисловість,46.0,8310.0,5272.0,5102.0,0.0,1246.0,0.0,4538.0,5.0
3,2012,Сфера послуг,27.0,161.0,463.0,1861.0,119.0,78.0,0.0,2326.0,2.0
4,2012,Сільське господарство,20.0,14.0,153.0,312.0,20.0,1364.0,0.0,312.0,0.3
5,2012,Транспорт,0.0,12.0,2050.0,750.0,48.0,8588.0,0.0,0.0,0.0
6,2035,Всього,5695.0,5419.0,7615.0,6457.0,10964.0,5873.0,657.0,6435.0,3904.0
7,2035,Домогосподарства,137.0,395.0,5450.0,1325.0,2251.0,0.0,642.0,2867.0,1739.0
8,2035,Промисловість,478.0,4910.0,1552.0,3374.0,0.0,676.0,0.0,2090.0,1268.0
9,2035,Сфера послуг,440.0,100.0,295.0,967.0,1642.0,0.0,14.0,1231.0,747.0


In [18]:
def reshape_df(d):
    new = d.pivot('year', 'sphere', 'k_t_ne').reset_index()
    new['source'] = d.source.iloc[0]
    new['is_vde'] = d.is_vde.iloc[0]
    return new.reindex(['year', 'source', 'is_vde', 'Всього', 'Промисловість', 'Домогосподарства', 'Транспорт',
       'Сфера послуг', 'Сільське господарство'], axis=1)

In [21]:
df.groupby(['year', 'source'], as_index=False).apply(reshape_df
 ).reset_index(drop=True
 ).fillna(0
#  ).to_csv('data_by_source.csv', index=False)

In [15]:
pd.read_csv('data_by_source.csv')

,year,source,Всього,Промисловість,Домогосподарства,Транспорт,Сфера послуг,Сільське господарство
0,2012,Біопаливо та відходи,1029.0,46.0,936.0,0.0,27.0,20.0
1,2012,Вугілля,9212.0,8310.0,715.0,12.0,161.0,14.0
2,2012,Газ,21698.0,5272.0,13760.0,2050.0,463.0,153.0
3,2012,Електроенергія,11130.0,5102.0,3105.0,750.0,1861.0,312.0
4,2012,Електроенергія з ВДЕ,710.0,0.0,198.0,48.0,119.0,20.0
5,2012,Нафта та нафтопродукти,11347.0,1246.0,71.0,8588.0,78.0,1364.0
6,2012,Сонячна енергія,0.0,0.0,0.0,0.0,0.0,0.0
7,2012,Теплова енергія,11853.0,4538.0,4677.0,0.0,2326.0,312.0
8,2012,Теплова енергія з ВДЕ,12.3,5.0,5.0,0.0,2.0,0.3
9,2035,Біопаливо та відходи,5695.0,478.0,137.0,3521.0,440.0,1119.0


In [29]:
order = df[['source', 'is_vde']].drop_duplicates(
                               ).reset_index(
                               ).sort_values('index', ascending=False
                               ).reset_index(drop=True)

In [38]:
order = dict(map(lambda v: (v[1], {'index': v[0], 'is_vde': bool(v[2])}), order.values))

In [40]:
with open('sources_order.json', 'w') as f:
    json.dump(order, f, allow_nan=False, ensure_ascii=False)

In [47]:
measures = {k.capitalize(): v for k, v in measures.items()}

In [48]:
with open('measures.json', 'w') as f:
    json.dump(measures, f, allow_nan=False, ensure_ascii=False)

## new data

In [15]:
df = pd.read_csv('data/data_from_report.csv', sep='\t')
df.columns = ['source', 'sphere', 'scenario', 'Y2012', 'Y2015', 'Y2020', 'Y2025', 'Y2030',
       'Y2035', 'Y2040', 'Y2045', 'Y2050']
df = pd.wide_to_long(df, stubnames='Y', i=['source', 'sphere', 'scenario'], j='year').reset_index()
df.columns = ['source', 'sphere', 'scenario', 'year', 'ktne']

df = df.loc[df.source != 'Всього'].copy()

In [17]:
totals = df.loc[df.sphere == 'загалом'].copy()
df = df.loc[df.sphere != 'загалом'].copy()

new_total = df.groupby(['source', 'scenario', 'year'], as_index=False).ktne.sum()
new_total['sphere'] = 'загалом'
df = pd.concat([df, new_total], ignore_index=True, sort=False)
# df.to_csv('data_report_longest.csv', index=False)

In [6]:
df = pd.read_csv('data/data_report_longest.csv')
df = df.loc[df.scenario != 'Ліберальний']

In [7]:
df.sort_values('ktne', ascending=False)

,source,sphere,scenario,year,ktne
1205,Електроенергія з ВДЕ,загалом,Революційний,2050,25292.0
1125,Газ,загалом,Базовий,2012,21698.0
1143,Газ,загалом,Революційний,2012,21698.0
1133,Газ,загалом,Базовий,2050,20578.0
1132,Газ,загалом,Базовий,2045,20285.0
1131,Газ,загалом,Базовий,2040,19486.0
1160,Електроенергія,загалом,Базовий,2050,18991.0
1159,Електроенергія,загалом,Базовий,2045,18605.0
1130,Газ,загалом,Базовий,2035,18518.0
1158,Електроенергія,загалом,Базовий,2040,17921.0


In [5]:
df.loc[739,]

source      Електроенергія
sphere       промисловість
scenario      Революційний
year                  2015
ktne                 10233
by_vde                dirt
Name: 739, dtype: object

In [8]:
is_vde = dict(zip(
    df.source.drop_duplicates().values,
    ['vde', 'vde', 'vde', 'dirt', 'dirt', 'dirt', 'dirt', 'dirt', 'vde',]
))
# df.loc[1171, 'ktne'] = 10233
# df.loc[739, 'ktne'] = 10233
df['by_vde'] = df.source.apply(lambda s: is_vde[s])
df_dy_vde = df.groupby(['sphere', 'scenario', 'year', 'by_vde',], as_index=False
          ).ktne.sum()

df_dy_vde.to_csv('data/by_vde.csv', index=False)

In [90]:
df.loc[(df.scenario == 'Революційний') & (df.year == 2015) & (df.sphere == 'промисловість')]

,source,sphere,scenario,year,ktne,by_vde
703,Біопаливо та відходи,промисловість,Революційний,2015,86.0,vde
712,Теплова енергія з ВДЕ,промисловість,Революційний,2015,10.0,vde
721,Електроенергія з ВДЕ,промисловість,Революційний,2015,232.0,vde
730,Вугілля,промисловість,Революційний,2015,5569.0,dirt
739,Електроенергія,промисловість,Революційний,2015,10233.0,dirt
748,Газ,промисловість,Революційний,2015,2762.0,dirt
757,Теплова енергія,промисловість,Революційний,2015,2870.0,dirt
766,Нафта та нафтопродукти,промисловість,Революційний,2015,814.0,dirt


In [9]:
def reshape_df(d):
    new = d.pivot('year', 'sphere', 'ktne').reset_index()
    new['by_vde'] = d.by_vde.iloc[0]
    new['scenario'] = d.scenario.iloc[0]
    return new

vde_wide = df_dy_vde.groupby(['year', 'by_vde', 'scenario'], as_index=False
          ).apply(reshape_df
          ).reset_index(drop=True
          ).fillna(0)

vde_wide.columns = [s.capitalize() if (i > 0 and i < 7) else s for i, s in enumerate(vde_wide.columns)]
vde_wide.to_csv('data/by_vde_wide.csv', sep=',', index=False)

In [92]:
df_dy_vde.sort_values('ktne', ascending=False)

,sphere,scenario,year,by_vde,ktne
16,загалом,Базовий,2050,dirt,80543.0
14,загалом,Базовий,2045,dirt,78298.0
12,загалом,Базовий,2040,dirt,75047.0
10,загалом,Базовий,2035,dirt,70869.0
8,загалом,Базовий,2030,dirt,65597.0
0,загалом,Базовий,2012,dirt,65241.0
18,загалом,Революційний,2012,dirt,65240.0
6,загалом,Базовий,2025,dirt,61018.0
4,загалом,Базовий,2020,dirt,55350.0
22,загалом,Революційний,2020,dirt,49923.0


In [10]:
def reshape_df(d):
    new = d.pivot('year', 'sphere', 'ktne').reset_index()
    new['source'] = d.source.iloc[0]
    new['scenario'] = d.scenario.iloc[0]
    return new

df.groupby(['year', 'source', 'scenario'], as_index=False
          ).apply(reshape_df
          ).reset_index(drop=True
          ).fillna(0
          ).to_csv('data/data_report_wide.csv', sep=',', index=False)

/home/nr/ongoing/ecoaction/env/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:1142: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis, keys=keys)


In [25]:
df.groupby(['year', 'source', 'scenario'], as_index=False
          ).apply(reshape_df
          ).reset_index(drop=True
          ).fillna(0)

,scenario,source,year,загалом,населення,промисловість,сфера послуг,сільське господарство,транспорт
0,Базовий,Біопаливо та відходи,2012,1029.0,936.0,46.0,27.0,20.0,0.0
1,Революційний,Біопаливо та відходи,2012,1029.0,936.0,46.0,27.0,20.0,0.0
2,Базовий,Вугілля,2012,9212.0,715.0,8310.0,161.0,14.0,12.0
3,Революційний,Вугілля,2012,9212.0,715.0,8310.0,161.0,14.0,12.0
4,Базовий,Газ,2012,21698.0,13760.0,5272.0,463.0,153.0,2050.0
5,Революційний,Газ,2012,21698.0,13760.0,5272.0,463.0,153.0,2050.0
6,Базовий,Електроенергія,2012,11131.0,3105.0,5102.0,1862.0,312.0,750.0
7,Революційний,Електроенергія,2012,11130.0,3105.0,5102.0,1861.0,312.0,750.0
8,Базовий,Електроенергія з ВДЕ,2012,709.0,198.0,325.0,118.0,20.0,48.0
9,Революційний,Електроенергія з ВДЕ,2012,710.0,198.0,325.0,119.0,20.0,48.0


In [11]:
df = pd.read_csv('data/data_report_wide.csv')

In [12]:
df.columns = [s.capitalize() if i > 2 else s for i, s in enumerate(df.columns)]
df.to_csv('data/data_report_wide.csv', sep=',', index=False)

In [34]:
df['Загалом'].idxmax()

37

In [65]:
df.loc[37, ]

source      Електроенергія
sphere       промисловість
scenario           Базовий
year                  2015
ktne                  4065
Name: 37, dtype: object

### parse costs

In [80]:
import re

In [93]:
with open('costs_to_parse.txt') as f:
    costs = f.read()

costs = re.sub('\s{2,}|\s+(?<=\d)|\s+(?=\d)', '\n', costs)

scens = re.split('\s*-{3,}\s*', costs)

snames = ['Консервативний', 'Ліберальний', 'Революційний']

sdfs = [];

for i, s in enumerate(scens):
    sdf = pd.DataFrame(list(map(lambda r: r.strip().split('\n'), re.findall('[^0-9]+[^А-яЄєЇїІіҐґ]+', s))))
    sdf['scenario'] = snames[i]
    sdfs.append(sdf)
    
sdf = pd.concat(sdfs, ignore_index=True, sort=False
       ).replace({'-': 0})

sdf.columns = ['action', 'Y2012', 'Y2015', 'Y2020', 'Y2025', 'Y2030', 'Y2035', 'Y2040', 'Y2045', 'Y2050', 'scenario']

sdf = pd.wide_to_long(sdf, stubnames='Y', i=['action', 'scenario',], j='year').reset_index()

sdf.columns = ['action', 'scenario', 'year', 'm_euro']
# sdf.to_csv('costs.csv', index=False)

In [17]:
df = pd.read_csv('data/costs.csv')
df = df.loc[(df.scenario != 'Ліберальний')]

new_actions = [
    'Вартість палива',
    'Капітальні інвестиції',
    'Капітальні інвестиції',
    'Капітальні інвестиції',
    'Транспортування, постачання та проміжні технології',
    'Експлуатаційні витрати',
    'Експлуатаційні витрати',
    'Субсидії («зелений» тариф)',
]

new_actions = dict(zip(df.action.drop_duplicates().values, new_actions))
df.action = df.action.apply(lambda a: new_actions[a])

df = df.groupby(['action', 'scenario', 'year',], as_index=False).m_euro.sum()

# df.to_csv('data/costs_agg.csv', index=False)

def groupcosts(d):
    res = d.pivot('scenario', 'year', 'm_euro')
    res['action'] = d.iloc[0, 0]
    return res

df.groupby(['action', 'scenario',], as_index=False
          ).apply(groupcosts
          ).reset_index(1
          ).drop(2012, axis=1
          ).to_csv('data/costs_agg_wide.csv', index=False)

In [20]:
len('енергоспоживання')

16

In [12]:
new_actions

{'Вартість палива': 'Вартість палива',
 'Нові капітальні інвестиції в технології виробництва е/е та тепла': 'Капітальні інвестиції',
 'Нові капітальні інвестиції в технології кінцевого енергоспоживання': 'Капітальні інвестиції',
 'Нові капітальні інвестиції в проміжні технології': 'Капітальні інвестиції',
 'Витрати на транспортування, постачання та проміжні технології': 'Витрати на транспортування, постачання та проміжні технології',
 'Експлуатаційні витрати (технології виробництва е/е та тепла)': 'Експлуатаційні витрати',
 'Експлуатаційні витрати (технології кінцевого енергоспоживання)': 'Експлуатаційні витрати',
 'Субсидії («зелений» тариф)': 'Субсидії («зелений» тариф)'}

In [15]:
df.columns

Index(['action', 'scenario', 'year', 'm_euro'], dtype='object')

,action,scenario,year,m_euro
0,Вартість палива,Консервативний,2015,13507
1,Вартість палива,Консервативний,2050,20803
2,Вартість палива,Революційний,2015,13500
3,Вартість палива,Революційний,2050,4123
4,"Витрати на транспортування, постачання та пром...",Консервативний,2015,1328
5,"Витрати на транспортування, постачання та пром...",Консервативний,2050,10085
6,"Витрати на транспортування, постачання та пром...",Революційний,2015,1307
7,"Витрати на транспортування, постачання та пром...",Революційний,2050,4798
8,Експлуатаційні витрати,Консервативний,2015,4659
9,Експлуатаційні витрати,Консервативний,2050,36497


## eresources

In [118]:
df = pd.read_excel('data/eresources.xlsx'
                  ).melt(id_vars=['source', 'scenario'],
                         value_vars=list(range(2015, 2055, 5)),
                         var_name='year',
                         value_name='Базовий',
                  ).sort_values(['scenario', 'year', 'source']
                  ).reset_index(drop=True)

df['Революційний'] = df.loc[int(len(df) / 2):, 'Базовий'].reset_index(drop=True)
df.source = df.source.str.strip()
df.dropna(axis=0).drop('scenario', axis=1).to_csv('data/eresources_long.csv', index=False)

In [121]:
df = pd.read_csv('data/eresources_long.csv')

In [25]:
df = pd.read_excel('data/eresources.xlsx'
                  ).melt(id_vars=['source', 'scenario'],
                         value_vars=list(range(2015, 2055, 5)),
                         var_name='year',
                         value_name='ktne',
                  ).sort_values(['scenario', 'year', 'source']
                  ).reset_index(drop=True)

def widen(d):
    new = d[['source', 'ktne']].set_index('source').T
    new['year'] = d.year.iloc[0]
    new['scenario'] = d.scenario.iloc[0]
    return new
df.groupby(['scenario', 'year'], as_index=False
          ).apply(widen
          ).reset_index(drop=True
          ).to_csv('data/eresources_wide.csv', index=False)

In [26]:
pd.read_csv('data/eresources_wide.csv')

,Імпорт,Біопаливо та відходи,Всього,Вугілля,Вітер,Газ,Гідро,Нафта,Ресурс для АЕС,Сонце,year,scenario
0,46899,1465,89519,27344,94,26055,464,10551,22985,40,2015,Базовий
1,45150,2112,116238,50042,258,33944,907,12252,16951,24,2020,Базовий
2,40365,2205,129515,60980,370,36513,1030,13126,15544,48,2025,Базовий
3,43767,2221,139172,68974,378,38546,1117,14229,13989,72,2030,Базовий
4,47573,2520,149989,75464,387,41036,1123,15340,14429,82,2035,Базовий
5,52237,2828,157233,80993,396,43660,1130,16078,12537,42,2040,Базовий
6,54353,3399,163762,86010,396,43810,1133,16837,12554,42,2045,Базовий
7,58739,3944,168245,90027,430,42761,1140,17753,12554,42,2050,Базовий
8,46899,1465,89519,27344,94,26055,464,10551,22985,40,2015,Революційний
9,40603,5857,104419,35081,772,33355,905,11293,16951,252,2020,Революційний


In [112]:
df.loc[~df.source.str.match('Імпор|Всього')].groupby(['year'], as_index=False).sum().max()

year              2050.0
Базовий         168651.0
Революційний    104466.0
dtype: float64

In [113]:
df.loc[df.year == 2015].groupby('source', as_index=False
           )['Базовий'].sum().sort_values('Базовий', ascending=False
                      )['source'].values.tolist()

['Всього',
 'Імпорт',
 'Вугілля',
 'Газ',
 'Ресурс для АЕС',
 'Нафта',
 'Біопаливо та відходи',
 'Гідро',
 'Вітер',
 'Сонце']